In [1]:
import re
import pandas as pd
import numpy as np
import sys

sys.path.append('./src')

In [2]:
%ls -lh data/features/assessments

ls: data/features/assessments: No such file or directory


In [7]:
train = read_from_clean('train.ftr')
test = read_from_clean('test.ftr')


Reading data/clean/train.ftr
Reading data/clean/test.ftr


In [3]:
from utils.common import prefix_list
from utils.io import read_from_clean, read_features
from utils.encoding import build_one_hot_encoder, get_categories

In [9]:
import re
import numpy as np
import pandas as pd

from utils.common import remove_dir_ext, prefix_list
from utils.io import read_from_clean, save_features
from utils.dataframe import apply_funcs, concat_dfs
from utils.encoding import build_one_hot_encoder, get_categories
from features.funcs import (filter_assessment_attempt,
                            assign_attempt_result,
                            calc_attempt_stats,
                            cum_by_user)






encoder = build_one_hot_encoder(train, test, ['title'])
funcs = [
    filter_assessment_attempt,
    assign_attempt_result,
    lambda df: calc_attempt_stats(df, keep_title=True),
]

stats = apply_funcs(train, funcs)

In [10]:
stats

,installation_id,game_session,title,num_correct,num_incorrect,attempts,accuracy,accuracy_group
0,0006a69f,901acc108f55a5a1,Mushroom Sorter (Assessment),1,0,1,1.000000,3
1,0006a69f,77b8ee947eb84b4e,Bird Measurer (Assessment),0,11,11,0.000000,0
2,0006a69f,6bdf9623adc94d89,Mushroom Sorter (Assessment),1,0,1,1.000000,3
3,0006a69f,9501794defd84e4d,Mushroom Sorter (Assessment),1,1,2,0.500000,2
4,0006a69f,a9ef3ecb3d1acc6a,Bird Measurer (Assessment),1,0,1,1.000000,3
...,...,...,...,...,...,...,...,...
17685,ffc90c32,460e8bdc2822b340,Chest Sorter (Assessment),1,0,1,1.000000,3
17686,ffd2871d,b05a02b52d5c1f4c,Cauldron Filler (Assessment),1,0,1,1.000000,3
17687,ffeb0b1b,dadd1a4d8ac68ab0,Cauldron Filler (Assessment),1,2,3,0.333333,1
17688,ffeb0b1b,a6885ab824fbc32c,Mushroom Sorter (Assessment),0,1,1,0.000000,0


In [36]:

def best_accuracy_so_far(df, encoder):
    """
    Calculate product of assessment stats and one-hot encoded title vector

    Input DataFrame:
            num_correct
    game_A            1
    game_B            2

    Output DataFrame:
            game_A_num_correct  game_B_num_correct
    game_A                   1                   0
    game_B                   0                   2
    """
    columns = get_categories(encoder)
    columns = [re.sub(r'[^a-zA-Z\d]+', '_', c).strip('_') for c in columns]
    columns = prefix_list(columns, 'acg')

    onehot = encoder.transform(df[['title']]).toarray().astype(np.int8)
    expanded = pd.DataFrame(df[['accuracy_group']].values * onehot, columns=columns)
    expanded = concat_dfs([df, expanded], axis=1)
    def func(gdf):
        return concat_dfs([gdf.drop(columns, axis=1), gdf[columns].cummax().shift(1)], axis=1)

    expanded = expanded.groupby('installation_id', sort=False).apply(func)
    return expanded.assign(best_acg=np.sum(expanded[columns].values * onehot, axis=1))


encoder = build_one_hot_encoder(stats, stats, ['title'])

best_accuracy_so_far(stats, encoder)

,installation_id,game_session,title,num_correct,num_incorrect,attempts,accuracy,accuracy_group,acg_Bird_Measurer_Assessment,acg_Cart_Balancer_Assessment,acg_Cauldron_Filler_Assessment,acg_Chest_Sorter_Assessment,acg_Mushroom_Sorter_Assessment,best_acg
0,0006a69f,901acc108f55a5a1,Mushroom Sorter (Assessment),1,0,1,1.000000,3,NaN,NaN,NaN,NaN,NaN,NaN
1,0006a69f,77b8ee947eb84b4e,Bird Measurer (Assessment),0,11,11,0.000000,0,0.0,0.0,0.0,0.0,3.0,0.0
2,0006a69f,6bdf9623adc94d89,Mushroom Sorter (Assessment),1,0,1,1.000000,3,0.0,0.0,0.0,0.0,3.0,3.0
3,0006a69f,9501794defd84e4d,Mushroom Sorter (Assessment),1,1,2,0.500000,2,0.0,0.0,0.0,0.0,3.0,3.0
4,0006a69f,a9ef3ecb3d1acc6a,Bird Measurer (Assessment),1,0,1,1.000000,3,0.0,0.0,0.0,0.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17685,ffc90c32,460e8bdc2822b340,Chest Sorter (Assessment),1,0,1,1.000000,3,3.0,3.0,3.0,0.0,3.0,0.0
17686,ffd2871d,b05a02b52d5c1f4c,Cauldron Filler (Assessment),1,0,1,1.000000,3,NaN,NaN,NaN,NaN,NaN,NaN
17687,ffeb0b1b,dadd1a4d8ac68ab0,Cauldron Filler (Assessment),1,2,3,0.333333,1,NaN,NaN,NaN,NaN,NaN,NaN
17688,ffeb0b1b,a6885ab824fbc32c,Mushroom Sorter (Assessment),0,1,1,0.000000,0,0.0,0.0,1.0,0.0,0.0,0.0
